In [102]:
import plotly.express as px
import numpy as np
import pandas as pd
from glob import glob
from pathlib import Path
from IPython.display import display, HTML
import plotly.graph_objects as go
import json
import os
import time

In [103]:
logs_path = Path("logs")

# Standalone

In [104]:
logs_standalone = logs_path / "standalone.logs.json"

In [105]:
logs_standalone_df = pd.read_json(logs_standalone)
logs_standalone_df

,epoch,step,absolut_step,loss_d,loss_g,time_elapsed,start_time,end_time,fid_score,inception_score
0,0,0,0,1.611353,37.177753,0.548701,2024-05-13 17:03:49.561235905,2024-05-13 17:03:50.109936953,391.634857,1.065864
1,1,0,1,0.374635,53.654175,0.159144,2024-05-13 17:03:53.888676167,2024-05-13 17:03:54.047820091,NaN,NaN
2,2,0,2,1.416522,39.805294,0.124094,2024-05-13 17:03:54.135853052,2024-05-13 17:03:54.259947062,NaN,NaN
3,3,0,3,0.353826,20.394459,0.113435,2024-05-13 17:03:54.342012882,2024-05-13 17:03:54.455448151,NaN,NaN
4,4,0,4,43.461674,25.461288,0.124356,2024-05-13 17:03:54.536193131,2024-05-13 17:03:54.660549164,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
95,95,0,95,2.848750,6.521394,0.112042,2024-05-13 17:04:51.291045189,2024-05-13 17:04:51.403087139,NaN,NaN
96,96,0,96,1.141844,5.678805,0.136434,2024-05-13 17:04:51.484804153,2024-05-13 17:04:51.621237993,NaN,NaN
97,97,0,97,2.495184,6.439551,0.136139,2024-05-13 17:04:51.719119072,2024-05-13 17:04:51.855258226,NaN,NaN
98,98,0,98,0.675325,3.769869,0.129866,2024-05-13 17:04:51.948642969,2024-05-13 17:04:52.078509092,NaN,NaN


In [106]:
px.line(logs_standalone_df[["absolut_step", "loss_g", "loss_d"]], x="absolut_step", y=["loss_g", "loss_d"], title="Losses")

In [107]:
px.line(logs_standalone_df[["absolut_step", "fid_score"]].dropna(), x="absolut_step", y="fid_score", title="Scores")

In [108]:
px.line(logs_standalone_df[["absolut_step", "inception_score"]].dropna(), x="absolut_step", y="inception_score", title="Scores")

# Distributed

In [94]:
logs_files = glob(str(logs_path / "*.logs.json"))
logs_files.pop(logs_files.index(str(logs_standalone)))
logs_files.sort()
logs_files

['logs/server_0.logs.json',
 'logs/worker_1.logs.json',
 'logs/worker_2.logs.json',
 'logs/worker_3.logs.json',
 'logs/worker_4.logs.json',
 'logs/worker_5.logs.json']

In [72]:
server_logs = logs_files.pop(0)
server_logs, logs_files

('logs/server_0.logs.json',
 ['logs/worker_1.logs.json',
  'logs/worker_2.logs.json',
  'logs/worker_3.logs.json',
  'logs/worker_4.logs.json',
  'logs/worker_5.logs.json'])

# Server

In [73]:
server_df = pd.read_json(server_logs, orient="index")
server_df["fid_score"] = server_df["fid_score"].apply(lambda x: 0 if x is None else x)
server_df["inception_score"] = server_df["inception_score"].apply(lambda x: 0 if x is None else x)
server_df["name"] = "server"
server_with_score_df = server_df[server_df["fid_score"].notnull() & server_df["inception_score"].notnull()]
display(server_df)
display(server_with_score_df)

,end_time,elapsed_time,logging_time_end,received_time,sent_time,epoch,start_time,is_calculation_time_start,inception_score,is_calculation_time_end,fid_calculation_time_start,fid_score,fid_calculation_time_end,name
0,2024-05-13 16:56:55.368268013,2.631052,1.715619e+09,2024-05-13 16:56:54.709145069,2024-05-13 16:56:52.885792971,0,2024-05-13 16:56:52.737215996,1.715619e+09,1.056370,1.715619e+09,1.715619e+09,452.766876,1.715619e+09,server
1,2024-05-13 16:56:59.667267084,0.885532,1.715619e+09,2024-05-13 16:56:59.592746019,2024-05-13 16:56:58.787747145,1,2024-05-13 16:56:58.781734943,NaN,NaN,NaN,NaN,NaN,NaN,server
2,2024-05-13 16:57:00.315038919,0.647096,1.715619e+09,2024-05-13 16:57:00.281295061,2024-05-13 16:56:59.671033144,2,2024-05-13 16:56:59.667943001,NaN,NaN,NaN,NaN,NaN,NaN,server
3,2024-05-13 16:57:00.985203981,0.669307,1.715619e+09,2024-05-13 16:57:00.960901022,2024-05-13 16:57:00.320356131,3,2024-05-13 16:57:00.315896988,NaN,NaN,NaN,NaN,NaN,NaN,server
4,2024-05-13 16:57:01.658004999,0.672470,1.715619e+09,2024-05-13 16:57:01.633717060,2024-05-13 16:57:00.987265110,4,2024-05-13 16:57:00.985534906,NaN,NaN,NaN,NaN,NaN,NaN,server
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,2024-05-13 16:58:56.203507900,0.703884,1.715620e+09,2024-05-13 16:58:56.154509068,2024-05-13 16:58:55.501394032,108,2024-05-13 16:58:55.499624014,NaN,NaN,NaN,NaN,NaN,NaN,server
109,2024-05-13 16:58:56.862545013,0.657084,1.715620e+09,2024-05-13 16:58:56.833885908,2024-05-13 16:58:56.209648848,109,2024-05-13 16:58:56.205461025,NaN,NaN,NaN,NaN,NaN,NaN,server
110,2024-05-13 16:58:57.534639120,0.670722,1.715620e+09,2024-05-13 16:58:57.510452032,2024-05-13 16:58:56.865885973,110,2024-05-13 16:58:56.863917112,1.715620e+09,1.206814,1.715620e+09,1.715620e+09,414.299011,1.715620e+09,server
111,2024-05-13 16:59:02.465160847,0.609760,1.715620e+09,2024-05-13 16:59:02.407190084,2024-05-13 16:59:01.859165907,111,2024-05-13 16:59:01.855401039,NaN,NaN,NaN,NaN,NaN,NaN,server


,end_time,elapsed_time,logging_time_end,received_time,sent_time,epoch,start_time,is_calculation_time_start,inception_score,is_calculation_time_end,fid_calculation_time_start,fid_score,fid_calculation_time_end,name
0,2024-05-13 16:56:55.368268013,2.631052,1.715619e+09,2024-05-13 16:56:54.709145069,2024-05-13 16:56:52.885792971,0,2024-05-13 16:56:52.737215996,1.715619e+09,1.056370,1.715619e+09,1.715619e+09,452.766876,1.715619e+09,server
10,2024-05-13 16:57:05.765360117,0.683631,1.715619e+09,2024-05-13 16:57:05.731727123,2024-05-13 16:57:05.083806038,10,2024-05-13 16:57:05.081728935,1.715619e+09,1.100727,1.715619e+09,1.715619e+09,408.924500,1.715619e+09,server
20,2024-05-13 16:57:17.192725897,0.749319,1.715619e+09,2024-05-13 16:57:17.091354132,2024-05-13 16:57:16.445441008,20,2024-05-13 16:57:16.443407059,1.715619e+09,1.036813,1.715619e+09,1.715619e+09,525.962891,1.715619e+09,server
30,2024-05-13 16:57:27.785007000,0.615499,1.715619e+09,2024-05-13 16:57:27.755499125,2024-05-13 16:57:27.175738096,30,2024-05-13 16:57:27.169507980,1.715619e+09,1.169341,1.715619e+09,1.715619e+09,464.955566,1.715619e+09,server
40,2024-05-13 16:57:39.057957172,0.672270,1.715619e+09,2024-05-13 16:57:39.032361984,2024-05-13 16:57:38.387607098,40,2024-05-13 16:57:38.385687113,1.715619e+09,1.648211,1.715619e+09,1.715619e+09,437.323608,1.715619e+09,server
50,2024-05-13 16:57:49.732101917,0.670663,1.715619e+09,2024-05-13 16:57:49.707270861,2024-05-13 16:57:49.063223124,50,2024-05-13 16:57:49.061439037,1.715619e+09,1.462968,1.715619e+09,1.715619e+09,443.785919,1.715619e+09,server
60,2024-05-13 16:58:00.985358000,0.703543,1.715619e+09,2024-05-13 16:58:00.935754061,2024-05-13 16:58:00.283749819,60,2024-05-13 16:58:00.281815052,1.715619e+09,1.460029,1.715619e+09,1.715619e+09,415.833588,1.715619e+09,server
70,2024-05-13 16:58:12.555135965,0.675133,1.715619e+09,2024-05-13 16:58:12.530586004,2024-05-13 16:58:11.881762981,70,2024-05-13 16:58:11.880002975,1.715619e+09,1.513320,1.715619e+09,1.715619e+09,430.093109,1.715619e+09,server
80,2024-05-13 16:58:24.469179869,0.659939,1.715620e+09,2024-05-13 16:58:24.439525127,2024-05-13 16:58:23.813211918,80,2024-05-13 16:58:23.809241056,1.715620e+09,1.264514,1.715620e+09,1.715620e+09,405.627136,1.715620e+09,server
90,2024-05-13 16:58:35.634095192,0.691006,1.715620e+09,2024-05-13 16:58:35.588483095,2024-05-13 16:58:34.945381880,90,2024-05-13 16:58:34.943089008,1.715620e+09,1.196784,1.715620e+09,1.715620e+09,429.553436,1.715620e+09,server


In [63]:
figure = go.Figure()
normalized_fid = server_with_score_df.copy()
lineplot_fid = px.line(
    normalized_fid,
    x="epoch",
    y="fid_score",
    title="Server logs",
).show()
lineplot_is = px.line(
    normalized_fid,
    x="epoch",
    y="inception_score",
    title="Server logs",
).show()
duration_plot = px.bar(
    normalized_fid,
    x="epoch",
    y="elapsed_time",
    title="Server logs",
    text="elapsed_time",
    template="plotly_white"
).show()

In [12]:
px.timeline(
    server_df,
    x_start="start_time",
    x_end="end_time",
    title="Server logs",
    template="plotly_white"
)

# Clients

In [13]:
def read_client_json(file_path: str):
    flattened_local_epochs = []
    flattened_swap = []
    flattened_send = []
    for outer_key, inner_dict in json.loads(open(file_path, "r").read()).items():
        root_attributes = {k: v for k, v in inner_dict.items() if not isinstance(v, dict)}
        if "start_time_swap" in root_attributes:
            flattened_swap.append({
                "outer_epoch": outer_key,
                "start_time": root_attributes["start_time_swap"],
                "end_time": root_attributes["end_time_swap"],
            })
        if "start_time_send" in root_attributes:
            flattened_send.append({
                "outer_epoch": outer_key,
                "start_time": root_attributes["start_time_send"],
                "end_time": root_attributes["end_time_send"],
            })
        for inner_key, values_dict in inner_dict.items():
            if not isinstance(values_dict, dict):
                continue
            flattened_local_epochs.append({
                "outer_epoch": outer_key,
                "local_epoch": inner_key,
                **values_dict,
                **root_attributes
            })

    # Convert flattened data to DataFrame
    name = os.path.basename(file_path).split(".")[0]
    
    df_local_epochs = pd.DataFrame(flattened_local_epochs)
    df_local_epochs["name"] = name

    df_swap = pd.DataFrame(flattened_swap)
    df_swap["name"] = f"{name}_swap"

    df_send = pd.DataFrame(flattened_send)
    df_send["name"] = f"{name}_send"

    return df_local_epochs, df_swap, df_send

In [14]:
clients_df = pd.concat([pd.concat(read_client_json(file)) for file in logs_files])

clients_df["start_time"] = pd.to_datetime(clients_df["start_time"], unit="s")
clients_df["end_time"] = pd.to_datetime(clients_df["end_time"], unit="s")

server_to_client_df = server_df.copy()
server_to_client_df["name"] = "server_recv"
server_to_client_df["start_time"] = server_to_client_df["received_time"]
clients_df = pd.concat([clients_df, server_to_client_df])

server_to_client_df = server_df.copy()
server_to_client_df["name"] = "server_sent"
server_to_client_df["end_time"] = server_to_client_df["sent_time"]
clients_df = pd.concat([clients_df, server_to_client_df])

clients_df

,outer_epoch,local_epoch,d_loss_real,d_loss_fake,d_total_loss,elapsed_time,start_time,end_time,start_time_send,end_time_send,...,logging_time_end,received_time,sent_time,epoch,is_calculation_time_start,inception_score,is_calculation_time_end,fid_calculation_time_start,fid_score,fid_calculation_time_end
0,0,0,0.458506,0.928804,1.387310,1.247399,2024-05-13 16:42:50.737629890,2024-05-13 16:42:51.974203825,1.715619e+09,1.715619e+09,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,0.412562,0.693147,1.105709,0.348445,2024-05-13 16:42:56.086025000,2024-05-13 16:42:56.219267130,1.715619e+09,1.715619e+09,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0,0.332771,0.693923,1.026694,0.150170,2024-05-13 16:42:56.915130854,2024-05-13 16:42:56.935981989,1.715619e+09,1.715619e+09,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0,0.323174,0.693147,1.016321,0.180792,2024-05-13 16:42:57.597502947,2024-05-13 16:42:57.611941099,1.715619e+09,1.715619e+09,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0,0.313262,0.693147,1.006409,0.187893,2024-05-13 16:42:58.284208059,2024-05-13 16:42:58.310095072,1.715619e+09,1.715619e+09,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,NaN,NaN,NaN,NaN,NaN,0.714380,2024-05-13 16:47:33.159618139,2024-05-13 16:47:33.163618088,NaN,NaN,...,1.715619e+09,2024-05-13 16:47:33.827744961,2024-05-13 16:47:33.163618088,245.0,NaN,NaN,NaN,NaN,NaN,NaN
246,NaN,NaN,NaN,NaN,NaN,0.670768,2024-05-13 16:47:33.876946926,2024-05-13 16:47:33.879567862,NaN,NaN,...,1.715619e+09,2024-05-13 16:47:34.522380113,2024-05-13 16:47:33.879567862,246.0,NaN,NaN,NaN,NaN,NaN,NaN
247,NaN,NaN,NaN,NaN,NaN,0.742974,2024-05-13 16:47:34.550444126,2024-05-13 16:47:34.552102804,NaN,NaN,...,1.715619e+09,2024-05-13 16:47:35.230605125,2024-05-13 16:47:34.552102804,247.0,NaN,NaN,NaN,NaN,NaN,NaN
248,NaN,NaN,NaN,NaN,NaN,0.666377,2024-05-13 16:47:35.296792030,2024-05-13 16:47:35.300291061,NaN,NaN,...,1.715619e+09,2024-05-13 16:47:35.935008049,2024-05-13 16:47:35.300291061,248.0,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
only_workers = [f"worker_{i+1}" for i in range(len(logs_files))]
only_workers

['worker_1', 'worker_2', 'worker_3', 'worker_4', 'worker_5']

In [16]:
px.line(
    clients_df[clients_df["name"].isin(only_workers)],
    x="start_time",
    y="d_total_loss",
    color="name",
    title="Client logs",
    template="plotly_white"
)

In [17]:
clients_and_server_df = pd.concat(
    [
        clients_df[["start_time", "end_time", "name"]],
        server_df[["start_time", "end_time", "name"]],
    ]
)
clients_and_server_df

,start_time,end_time,name
0,2024-05-13 16:42:50.737629890,2024-05-13 16:42:51.974203825,worker_1
1,2024-05-13 16:42:56.086025000,2024-05-13 16:42:56.219267130,worker_1
2,2024-05-13 16:42:56.915130854,2024-05-13 16:42:56.935981989,worker_1
3,2024-05-13 16:42:57.597502947,2024-05-13 16:42:57.611941099,worker_1
4,2024-05-13 16:42:58.284208059,2024-05-13 16:42:58.310095072,worker_1
...,...,...,...
245,2024-05-13 16:47:33.159618139,2024-05-13 16:47:33.873998165,server
246,2024-05-13 16:47:33.876946926,2024-05-13 16:47:34.547715187,server
247,2024-05-13 16:47:34.550444126,2024-05-13 16:47:35.293417931,server
248,2024-05-13 16:47:35.296792030,2024-05-13 16:47:35.963169098,server


In [18]:
timeline = px.timeline(
    clients_and_server_df,
    x_start="start_time",
    x_end="end_time",
    color="name",
    title="Client logs",
    opacity=0.4,
    template="plotly_white"
)
# add vertical line to show when server logs started
for i in server_df["start_time"]:
    timeline.add_vline(x=i, line_dash="dash", line_color="red")
timeline.show()